In [17]:
import os 
SUBMIT_ARGS = "--packages graphframes:graphframes:0.7.0-spark2.4-s_2.11 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS
from spark import *
from graphframes import *
from pyspark import SparkContext
from pyspark.sql.functions import concat, col, lit
from pyspark.sql import *
from graphframes.examples import Graphs
path = "201508_trip_data.csv"
edges = sqlContext.read.format("csv").option("header", "true").load(path).withColumnRenamed("Start Terminal", "src").withColumnRenamed("End Terminal", "dst").dropDuplicates()

path2 = "201508_station_data.csv"
vertexs = sqlContext.read.format("csv").option("header", "true").load(path2).withColumnRenamed("station_id", "id").dropDuplicates()


vertexs = vertexs.withColumn('lat-long', concat(col('lat'), lit(' : '), col('long')))
vertexs = vertexs.drop('lat')
vertexs = vertexs.drop('long')

edges.write.csv('edges.csv')
vertexs.write.csv('vertexs.csv')


g = GraphFrame(vertexs, edges)


# Display the vertex and edge DataFrames
g.vertices.show()

+---+--------------------+---------+-------------+------------+--------------------+
| id|                name|dockcount|     landmark|installation|            lat-long|
+---+--------------------+---------+-------------+------------+--------------------+
| 38|       Park at Olive|       15|    Palo Alto|   8/14/2013|37.4256839 : -122...|
| 46|Washington at Kea...|       15|San Francisco|   8/19/2013|37.795425 : -122....|
| 37|Cowper at University|       11|    Palo Alto|   8/14/2013|37.448598 : -122....|
| 45|Commercial at Mon...|       15|San Francisco|   8/19/2013|37.794231 : -122....|
| 22|Redwood City Calt...|       25| Redwood City|   8/15/2013|37.486078 : -122....|
|  3|San Jose Civic Ce...|       15|     San Jose|    8/5/2013|37.330698 : -121....|
| 36|California Ave Ca...|       15|    Palo Alto|   8/14/2013|37.429082 : -122....|
| 77|   Market at Sansome|       27|San Francisco|   8/25/2013|37.789625 : -122....|
| 56|     Beale at Market|       19|San Francisco|   8/20/2013|37

In [21]:
g.edges.show()

+-------+--------+---------------+--------------------+---+---------------+--------------------+---+------+---------------+--------+
|Trip ID|Duration|     Start Date|       Start Station|src|       End Date|         End Station|dst|Bike #|Subscriber Type|Zip Code|
+-------+--------+---------------+--------------------+---+---------------+--------------------+---+------+---------------+--------+
| 912279|     221| 8/31/2015 9:43|Palo Alto Caltrai...| 34| 8/31/2015 9:47|Cowper at University| 37|   140|     Subscriber|   94107|
| 911099|    2995|8/29/2015 17:48|San Francisco Cal...| 69|8/29/2015 18:38|San Francisco Cal...| 70|   471|       Customer|   90803|
| 910741|     344| 8/29/2015 8:13|Grant Avenue at C...| 73| 8/29/2015 8:19|Harry Bridges Pla...| 50|   618|     Subscriber|   94133|
| 910677|    6876|8/28/2015 21:42|Embarcadero at Br...| 54|8/28/2015 23:36|Embarcadero at Br...| 54|   619|       Customer|   95035|
| 910651|     641|8/28/2015 20:40|     2nd at Townsend| 61|8/28/2015 

In [9]:
g.inDegrees.show()

+---+--------+
| id|inDegree|
+---+--------+
| 51|    1124|
|  7|     205|
| 54|     943|
| 11|     119|
| 69|    3761|
| 29|     130|
| 42|     949|
| 73|     685|
| 64|     809|
|  3|      62|
| 30|     156|
| 34|     160|
| 59|     470|
|  8|      68|
| 22|     148|
| 28|     676|
| 16|     114|
| 35|     148|
| 71|     576|
| 47|     723|
+---+--------+
only showing top 20 rows



In [5]:
g.outDegrees.show()

+---+---------+
| id|outDegree|
+---+---------+
| 51|     1246|
|  7|      179|
| 54|     1071|
| 11|      155|
| 69|     3483|
| 29|      123|
| 42|      891|
| 73|     1114|
| 64|     1168|
|  3|       94|
| 30|      154|
| 34|      206|
| 59|      572|
|  8|       55|
| 28|      712|
| 22|      228|
| 35|      138|
| 16|      130|
| 71|      850|
| 47|      680|
+---+---------+
only showing top 20 rows



In [20]:
motif=g.find("(vertex1)-[e]->(vertex2); (vertex2)-[e2]->(vertex1)")
motif=motif.dropDuplicates().show()

+--------------------+--------------------+--------------------+--------------------+
|             vertex1|                   e|             vertex2|                  e2|
+--------------------+--------------------+--------------------+--------------------+
|[48, Embarcadero ...|[910444, 6343, 8/...|[50, Harry Bridge...|[856625, 1717, 7/...|
|[74, Steuart at M...|[910173, 657, 8/2...|[69, San Francisc...|[867077, 630, 7/2...|
|[70, San Francisc...|[909537, 917, 8/2...|[74, Steuart at M...|[895031, 694, 8/1...|
|[45, Commercial a...|[908721, 1513, 8/...|[70, San Francisc...|[894203, 993, 8/1...|
|[70, San Francisc...|[907542, 778, 8/2...|[51, Embarcadero ...|[853214, 570, 7/2...|
|[70, San Francisc...|[907542, 778, 8/2...|[51, Embarcadero ...|[851622, 561, 7/1...|
|[25, Stanford in ...|[906569, 1015, 8/...|[22, Redwood City...|[894159, 642, 8/1...|
|[70, San Francisc...|[905058, 759, 8/2...|[76, Market at 4t...|[860439, 338, 7/2...|
|[63, Howard at 2n...|[904696, 369, 8/2...|[50, Harry 